In [41]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import desc
spark = SparkSession.builder.appName("xxx").getOrCreate()
spark_df = spark.read.csv("hdfs://localhost:9000/user/xxx/process_data.csv",header=True, inferSchema=True)

In [42]:
spark_df.show(10)

+-------------------+------------+-------------------+--------+--------------------+-----------+------+--------+--------------------+----+--------------------+--------+------+--------+--------+--------+--------+
|              时间0|      访问IP|              时间2|HTTP谓词|            访问目录|   HTTP协议|状态码|等待时间|            访问网址|标记|              浏览器|操作系统|  设备|是否电脑|是否爬虫|是否手机|是否平板|
+-------------------+------------+-------------------+--------+--------------------+-----------+------+--------+--------------------+----+--------------------+--------+------+--------+--------+--------+--------+
|2018-12-30 00:00:01|  127.0.0.1 |2018-12-30 00:00:01|    GET |                  / |  HTTP/1.1 |   200|    8259|                  - |   1|                curl|   Linux| Other|   false|   false|   false|   false|
|2018-12-30 00:00:13| 172.20.4.1 |2018-12-30 00:00:13|   POST |         /logout.do |  HTTP/1.1 |   200|    8259|  http://xxx.xxx...|   1|       Mobile Safari|     iOS|iPhone|   false|   false|    true|   

# 南丁而尔玫瑰图

In [20]:
spark_df.groupBy('访问网址').count().orderBy(desc('count')).show(30)
# pandas_df["访问网址"].value_counts()[:30]

/home/xxx/.local/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.


# 状态码图

In [44]:
spark_df.groupBy('状态码').count().orderBy(desc('count')).show()
# pandas_df["状态码"].value_counts()

+------+-------+
|状态码|  count|
+------+-------+
|   200|1124566|
|   404| 145718|
|   302|  29023|
|   500|   1260|
|   206|     64|
|   501|     12|
+------+-------+



# 每分钟访客量图

In [72]:
from pyspark.sql.functions import year, month, dayofmonth
from pyspark.sql.functions import hour, minute, second

from pyspark.sql.functions import split, explode, concat, concat_ws

Date = spark_df.select(year('时间0').alias('year'), 
          month('时间0').alias('month'),
          dayofmonth('时间0').alias('day'),
          hour('时间0').alias('hour'),
          minute('时间0').alias('minute'),
)

df_concat = Date.withColumn("Time", concat_ws("-",Date['year'], \
                                                       Date['month'], Date['day'], Date['hour'], Date['minute']))
for col in ["year","month","day","hour","minute"]:
    df_concat = df_concat.drop(col)    

df_time = df_concat.groupBy('Time').count()
df_time.toPandas().to_csv("data/time.csv",index=False)
df_time.show()

# import pandas as pd
# pandas_df = df.toPandas()
# pandas_df.index = pandas_df["时间0"]
# pandas_df.index = pd.to_datetime(pandas_df.index)
# index = pandas_df.index.to_period('T')
# index = index.value_counts()
# min_data = index.sort_index()
# min_data.to_csv("result/每分钟访客量.csv")

+----------------+-----+
|            Time|count|
+----------------+-----+
|  2018-12-30-2-1|   51|
|  2018-12-30-9-5|   55|
|2018-12-30-19-39|   58|
|2018-12-30-23-12|  136|
| 2018-12-31-0-51|    3|
| 2018-12-31-8-34|   87|
|2018-12-31-13-29|  121|
|2018-12-31-15-47|  117|
|2018-12-31-16-32|   58|
|2018-12-31-17-14|   48|
|   2019-1-1-1-20|   12|
|   2019-1-1-2-55|    4|
|   2019-1-1-6-50|    4|
|  2019-1-1-10-40|   30|
|  2019-1-1-11-41|    8|
|  2019-1-1-19-46|  177|
|  2019-1-1-20-51|   20|
|  2019-1-1-20-57|  149|
|  2019-1-1-22-42|   88|
|  2019-1-1-23-14|   11|
+----------------+-----+
only showing top 20 rows



In [78]:
spark_df.groupBy('浏览器').count().orderBy(desc('count')).show()
spark_df.groupBy('浏览器').count().orderBy(desc('count')).toPandas().to_csv("result/用户浏览器统计.csv",index=False)

+--------------------+------+
|              浏览器| count|
+--------------------+------+
|                  IE|277333|
|Chrome Mobile Web...|252583|
|   QQ Browser Mobile|201151|
|                Edge|199190|
|          QQ Browser| 89617|
|              Chrome| 53783|
|         MiuiBrowser| 47203|
|       Mobile Safari| 43923|
|      Sogou Explorer| 27550|
|          UC Browser| 22564|
|Mobile Safari UI/...| 21386|
|             Android| 20773|
|             Firefox| 14050|
|              Safari|  9592|
|       Chrome Mobile|  9516|
|    Samsung Internet|  3846|
|        MobileSafari|  2898|
|               Opera|   701|
|      Firefox Mobile|   698|
|              okhttp|   584|
+--------------------+------+
only showing top 20 rows



In [79]:
spark_df.groupBy('操作系统').count().orderBy(desc('count')).show()
spark_df.groupBy('操作系统').count().orderBy(desc('count')).toPandas().to_csv("result/用户操作系统统计.csv",index=False)

+--------+------+
|操作系统| count|
+--------+------+
| Windows|669463|
| Android|541473|
|     iOS| 78347|
|Mac OS X|  5365|
|   Linux|  5260|
|   Other|   735|
+--------+------+



In [80]:
spark_df.groupBy('设备').count().orderBy(desc('count')).show()
spark_df.groupBy('设备').count().orderBy(desc('count')).toPandas().to_csv("result/用户设备统计.csv",index=False)

+------------------+------+
|              设备| count|
+------------------+------+
|             Other|675445|
|            iPhone| 64522|
|            PACM00| 33019|
|            PADM00| 30187|
|          COL-AL10| 21974|
|         vivo X21A| 20945|
|          Oppo R11| 18442|
|       XiaoMi MI 8| 13582|
|Generic Smartphone| 12369|
|            PAAM00| 11681|
|          HWI-AL00| 11514|
|          EML-AL00| 10987|
|              iPad| 10528|
|      XiaoMi MI 6X|  9514|
|       XiaoMi MI 6|  9369|
|          Oppo R9s|  8686|
|          PAR-AL00|  8465|
|          ANE-AL00|  8256|
|           vivo X9|  7835|
|         Oppo R11s|  7326|
+------------------+------+
only showing top 20 rows

